# TF-IDF векторизация

TF-IDF - крайне важный метод векторизации текста. Он позволяет учитывать не только частоту токена в тексте, но и частоту его появления вообще в корпусе. С помощью данной векторизации можно рассчитывать важность слова для конкретного документа, учитывать, насколько токен делает данный документ уникальным относительно корпуса.

$$ TfIdf = TF * IDF $$

$$ TF - частота\ слова\ в\ документе $$

$$ IDF = log(\frac{|D|}{|D_{word}|}) + 1 $$

$$ |D|\ -\ количество\ документов\ в\ корпусе,\ |D_{word}|\ -\ количество\ документов\ из\ корпуса,\ в\ которых\ встретилось\ слово\ word$$

Рассмотрим реализацию Tfidf из библиотеки sklearn и посмотрим, насколько векторизация соотносится с формулой.

In [19]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

In [25]:
corpus = [
    'one two',
    'one',
    'four one four'
]

In [37]:
vect1 = TfidfVectorizer(smooth_idf=False)
X = vect1.fit_transform(corpus)
print(vect1.get_feature_names())
print(X.toarray())

['four', 'one', 'two']
[[0.         0.43016528 0.90275015]
 [0.         1.         0.        ]
 [0.97277169 0.23176546 0.        ]]


In [38]:
# one
one = (1 / 2) * (np.log(3 / 3) + 1)
one

0.5

In [39]:
# two
two = (1 / 2) * (np.log(3 / 1) + 1)
two

1.049306144334055

Не похоже на значения первого вектора. Нормализуем вектор.

In [40]:
one / (one ** 2 + two ** 2 + 0) ** 0.5

0.430165282498796

In [41]:
two / (one ** 2 + two ** 2 + 0) ** 0.5

0.9027501480103624

Таким образом, TfidfVectorizer считает значения по формуле и нормализует получившийся вектор.

Напишем что-то похожее. Будем считать, что мы произвели частотную векторизацию(что несложно сделать с помощью CountVectorizer, либо руками посчитать вхождение токенов в текст), и теперь у нас есть соответствующая матрица признаков.

In [80]:
X_train = np.arange(0, 200, 5).reshape(10, 4) % 6
X_train

array([[0, 5, 4, 3],
       [2, 1, 0, 5],
       [4, 3, 2, 1],
       [0, 5, 4, 3],
       [2, 1, 0, 5],
       [4, 3, 2, 1],
       [0, 5, 4, 3],
       [2, 1, 0, 5],
       [4, 3, 2, 1],
       [0, 5, 4, 3]])

In [109]:
class Tfidf:
    def fit(self, X):
        self.X_ = X
    def transform(self):
        self.tf = X_train / np.sum(X_train, axis=1)[:, None]
        self.idf = idf = np.log(X_train.shape[0] / np.where(X_train > 0, 1, 0).sum(axis=0)) + 1
        res = self.tf * self.idf
        return res / (np.sum(res ** 2, axis=1) ** 0.5)[:, None] 

In [110]:
vect = Tfidf()
vect.fit(X_train)
vect.transform()

array([[0.        , 0.62770758, 0.68127612, 0.37662455],
       [0.50980365, 0.1687169 , 0.        , 0.84358452],
       [0.82327577, 0.40868835, 0.36963816, 0.13622945],
       [0.        , 0.62770758, 0.68127612, 0.37662455],
       [0.50980365, 0.1687169 , 0.        , 0.84358452],
       [0.82327577, 0.40868835, 0.36963816, 0.13622945],
       [0.        , 0.62770758, 0.68127612, 0.37662455],
       [0.50980365, 0.1687169 , 0.        , 0.84358452],
       [0.82327577, 0.40868835, 0.36963816, 0.13622945],
       [0.        , 0.62770758, 0.68127612, 0.37662455]])

In [111]:
print(vect.tf, '\n')
print(vect.idf)

[[0.         0.41666667 0.33333333 0.25      ]
 [0.25       0.125      0.         0.625     ]
 [0.4        0.3        0.2        0.1       ]
 [0.         0.41666667 0.33333333 0.25      ]
 [0.25       0.125      0.         0.625     ]
 [0.4        0.3        0.2        0.1       ]
 [0.         0.41666667 0.33333333 0.25      ]
 [0.25       0.125      0.         0.625     ]
 [0.4        0.3        0.2        0.1       ]
 [0.         0.41666667 0.33333333 0.25      ]] 

[1.51082562 1.         1.35667494 1.        ]


Что еще может TfidfVectorizer(помимо основных метод из CountVectorizer)?

Можно изменить норму, с помощью которой мы нормируем вектора. Доступны нормы l1 и l2.

In [112]:
vect1 = TfidfVectorizer(smooth_idf=False, norm='l1')
X = vect1.fit_transform(corpus)
print(vect1.get_feature_names())
print(X.toarray())

['four', 'one', 'two']
[[0.         0.32272511 0.67727489]
 [0.         1.         0.        ]
 [0.80758961 0.19241039 0.        ]]


Можно учитывать только частоту вхождения слова в документ(только tf):

In [118]:
vect1 = TfidfVectorizer(smooth_idf=False, use_idf=False)
X = vect1.fit_transform(corpus)
print(vect1.get_feature_names())
print(X.toarray())

['four', 'one', 'two']
[[0.         0.70710678 0.70710678]
 [0.         1.         0.        ]
 [0.89442719 0.4472136  0.        ]]


Далее, можно варьировать формулу для TF-IDF. Во избежание деления на близкое к нулю значение в логарифме, можно использовать параметр smooth_idf(по умолчанию используется). В данном случае к числители и знаменателю в логарифме добавляется 1:
$$ IDF = log(\frac{|D| + 1}{|D_{word}| + 1}) + 1 $$

In [119]:
vect1 = TfidfVectorizer(smooth_idf=True, use_idf=False)
X = vect1.fit_transform(corpus)
print(vect1.get_feature_names())
print(X.toarray())

['four', 'one', 'two']
[[0.         0.70710678 0.70710678]
 [0.         1.         0.        ]
 [0.89442719 0.4472136  0.        ]]


И, наконец, можно уменьшить влияние частоты вхождения слова в документ:

In [121]:
vect1 = TfidfVectorizer(sublinear_tf=True)
X = vect1.fit_transform(corpus)
print(vect1.get_feature_names())
print(X.toarray())

['four', 'one', 'two']
[[0.         0.50854232 0.861037  ]
 [0.         1.         0.        ]
 [0.94420307 0.32936389 0.        ]]


In [125]:
corpus

['one two', 'one', 'four one four']

С использованием этого параметра tf заменяется на 1 + log(tf), и при больших значениях tf разница будет ощутимой.

Также в CountVectorizer и TfidfVectorizer можно учитывать n-граммы слов, учитывая n разных диапозонов.

Например, можно учитывать только биграммы:

In [145]:
corpus = [
    'one two four',
    'one five one five',
    'two six seven'
]

In [146]:
vect1 = TfidfVectorizer(ngram_range=(2, 2))
X = vect1.fit_transform(corpus)
print(vect1.get_feature_names())
print(X.toarray())

['five one', 'one five', 'one two', 'six seven', 'two four', 'two six']
[[0.         0.         0.70710678 0.         0.70710678 0.        ]
 [0.4472136  0.89442719 0.         0.         0.         0.        ]
 [0.         0.         0.         0.70710678 0.         0.70710678]]


Или юниграммы и биграммы:

In [148]:
vect1 = TfidfVectorizer(ngram_range=(1, 2))
X = vect1.fit_transform(corpus)
print(vect1.get_feature_names())
print(X.toarray())

['five', 'five one', 'four', 'one', 'one five', 'one two', 'seven', 'six', 'six seven', 'two', 'two four', 'two six']
[[0.         0.         0.49047908 0.37302199 0.         0.49047908
  0.         0.         0.         0.37302199 0.49047908 0.        ]
 [0.59460647 0.29730323 0.         0.45221354 0.59460647 0.
  0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.
  0.46735098 0.46735098 0.46735098 0.35543247 0.         0.46735098]]


Можно рассматривать не слова, а символы:

In [162]:
vect1 = TfidfVectorizer(analyzer='char')
X = vect1.fit_transform(corpus)
print(vect1.get_feature_names())
print(X.toarray())

[' f', ' o', ' s', ' t', 'e ', 'en', 'ev', 'fi', 'fo', 'iv', 'ix', 'ne', 'o ', 'on', 'ou', 'se', 'si', 'tw', 'ur', 've', 'wo', 'x ']
[[0.26807016 0.         0.         0.35248004 0.26807016 0.
  0.         0.         0.35248004 0.         0.         0.26807016
  0.26807016 0.26807016 0.35248004 0.         0.         0.26807016
  0.35248004 0.         0.26807016 0.        ]
 [0.31403645 0.20646009 0.         0.         0.47105467 0.
  0.         0.41292019 0.         0.41292019 0.         0.31403645
  0.         0.31403645 0.         0.         0.         0.
  0.         0.31403645 0.         0.        ]
 [0.         0.         0.56995099 0.         0.         0.2849755
  0.2849755  0.         0.         0.         0.2849755  0.
  0.21673121 0.         0.         0.2849755  0.2849755  0.21673121
  0.         0.21673121 0.21673121 0.2849755 ]]


И, наконец, можно рассматривать только n-граммы букв, что может быть полезно при важности учета каких-либо подстрок.

In [163]:
corpus = [
    'one two four',
    'one five one five',
    'two six seven'
]

In [164]:
vect1 = TfidfVectorizer(analyzer='char_wb', ngram_range=(2, 2))
X = vect1.fit_transform(corpus)
print(vect1.get_feature_names())
print(X.toarray())

[' f', ' o', ' s', ' t', 'e ', 'en', 'ev', 'fi', 'fo', 'iv', 'ix', 'n ', 'ne', 'o ', 'on', 'ou', 'r ', 'se', 'si', 'tw', 'ur', 've', 'wo', 'x ']
[[0.25066171 0.25066171 0.         0.25066171 0.25066171 0.
  0.         0.         0.32959003 0.         0.         0.
  0.25066171 0.25066171 0.25066171 0.32959003 0.32959003 0.
  0.         0.25066171 0.32959003 0.         0.25066171 0.        ]
 [0.28332116 0.28332116 0.         0.         0.56664232 0.
  0.         0.37253328 0.         0.37253328 0.         0.
  0.28332116 0.         0.28332116 0.         0.         0.
  0.         0.         0.         0.28332116 0.         0.        ]
 [0.         0.         0.53659627 0.20404765 0.         0.26829814
  0.26829814 0.         0.         0.         0.26829814 0.26829814
  0.         0.20404765 0.         0.         0.         0.26829814
  0.26829814 0.20404765 0.         0.20404765 0.20404765 0.26829814]]


## Применение на тексте

Применим данную векторизацию к тексту Ф.М.Достоевского "Идиот"

In [301]:
f = open('idiot.txt', 'r')
data = f.read()

Обучение на большом произведении занимает меньше секунды.

In [302]:
%%time
tiv = TfidfVectorizer()
# tiv.fit([data])

CPU times: user 50 µs, sys: 1e+03 ns, total: 51 µs
Wall time: 53.9 µs


Разобьем текст по строкам и отберем из них те, длина которых от 40 до 50 символов(чтобы слова были, но при этом не в огромном количестве).

In [303]:
samples = list(filter(lambda s: len(s) > 40 and len(s) < 50, data.split('\n')))
len(samples)

209

Векторизуем данный корпус с помощью TfidfVectorizer

In [304]:
tiv.fit(samples)
vect = tiv.transform(samples).toarray()
vect.shape

(209, 690)

Выберем какие-нибудь предложения и немного модифицируем их для тестовых объектов.

In [340]:
samples[30:35]

['— Как! — вскричал князь вне себя от удивления.',
 '— Настасью Филипповну, — пробормотал князь.',
 '— С пулями! — вскричала Настасья Филипповна.',
 '— Папенька, вас спрашивают, — крикнул Коля.',
 'Настасья Филипповна хохотала как в истерике.']

In [341]:
sent = ["Князь вскричал и показал себя, на лице его было удивление!",
        "Настасья Филипповна сказала, что это был князь",
        "Их пулями гнали из этих мест в истерике",
        "Князь Князь Князь",
        "князь Князь князь князь? князь! вот это князь",
        "Князь решил изучать нейросети"]
sent_corpus = tiv.transform(sent).toarray()

Посчитаем косинусные расстояния между тестовыми объектами и всеми объектами в корпусе. Во многих случаях будет происходить деление на близкое к нулю значение, возникнет много nan-ов, которые мы заменим на единицы(максимальное расстояние). 

In [342]:
from scipy.spatial import distance
distances = np.array([[distance.cosine(sent_vector, input_vect) for input_vect in vect]
                                                          for sent_vector in sent_corpus])
distances = np.nan_to_num(distances, nan=1)
distances.shape

(6, 209)

Выберем самые близкие по косинусному расстоянию вектора в корпусе:

In [339]:
min_ind = np.argmin(distances, axis=1)
print(min_ind)
[samples[ind] for ind in min_ind]

[ 30  32  32 185 125 206]


['— Как! — вскричал князь вне себя от удивления.',
 '— С пулями! — вскричала Настасья Филипповна.',
 '— С пулями! — вскричала Настасья Филипповна.',
 '— С Настасьей Филипповной! — вскричал князь.',
 '— Что это у вас? — спросил с беспокойством князь.',
 '— Н-ни за что! — решил князь: — ни-ни-ни!']

Получили предложения, похожие на тестовые. Таким образом, Tfidf-векторизация действительно неплохо работает с текстами.

Проделаем то же самое с CountVectorizer.

In [345]:
%%time
cv = CountVectorizer()
cv.fit(samples)
vect = cv.transform(samples).toarray()
vect.shape
sent = ["Князь вскричал и показал себя, на лице его было удивление!",
        "Настасья Филипповна сказала, что это был князь",
        "Их пулями гнали из этих мест в истерике",
        "Князь Князь Князь",
        "князь Князь князь князь? князь! вот это князь",
        "Князь решил изучать нейросети"]
sent_corpus = cv.transform(sent).toarray()

CPU times: user 11.6 ms, sys: 1.9 ms, total: 13.5 ms
Wall time: 13.4 ms


In [346]:
distances = np.array([[distance.cosine(sent_vector, input_vect) for input_vect in vect]
                                                          for sent_vector in sent_corpus])
distances = np.nan_to_num(distances, nan=1)
min_ind = np.argmin(distances, axis=1)
print(min_ind)
[samples[ind] for ind in min_ind]

[ 30 125  32  31  31  31]


['— Как! — вскричал князь вне себя от удивления.',
 '— Что это у вас? — спросил с беспокойством князь.',
 '— С пулями! — вскричала Настасья Филипповна.',
 '— Настасью Филипповну, — пробормотал князь.',
 '— Настасью Филипповну, — пробормотал князь.',
 '— Настасью Филипповну, — пробормотал князь.']

Видно, что вектора, полученные с помощью частотной векторизации, дают при сравнении другие результаты, причем менее точные хотя бы с точки зрения пересечения множеств слов. 